<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_2/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.
Probar términos de interés y explicar
similitudes en el espacio de embeddings.
Intentar plantear y probar tests de
analogías. Graficar los embeddings
resultantes.
Sacar conclusiones.

Algunos de los sitios de textos sugeridos fueron:
- [Project Gutenberg](https://www.gutenberg.org/browse/scores/top)
- [Textos.info](https://www.textos.info/)

Se eligió analizar los textos pertenecientes a [The Project Gutenberg eBook of The Adventures of Sherlock Holmes](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)


Por razones de compatibilidad con gensim, pandas y numpy hay que correr la siguiente linea de código.

In [1]:
!pip uninstall -y numpy gensim
!pip install --upgrade numpy gensim

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [2]:
import os
filename = 'dataset.txt'
text_path = 'https://raw.githubusercontent.com/jmestanza/natural-language-processing-practice/refs/heads/main/desafios/Desafio_2/the_adventures_of_sherlock_holmes.txt'
curr_dir = os.getcwd()
dataset_path = os.path.join(curr_dir, filename)
if not os.path.exists(dataset_path):
  !wget {text_path} -O {filename}

os.listdir(curr_dir)


['.config', 'dataset_clean.txt', 'dataset.txt', 'sample_data']

In [3]:
with open(dataset_path, 'r') as f:
  text = f.read()

print(text[:1500])

﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***




The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A 

## Limpieza de dataset
Debemos obtener solo los textos de adentro de los capítulos y subcapítulos.

Además debemos ignorar el texto que viene con la metadata de la obra (presente en el principio y final del dataset).

La estructura es


```
I. Chapter title
I.
text
II.
text
```



In [4]:
import re

# Read the text file
with open(dataset_path, "r", encoding="utf-8") as file:
    text = file.read()

# Remove everything before the actual content
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

# Keep only the content between the start and end markers
text = text.split(start_marker, 1)[-1].split(end_marker, 1)[0]

# Define a pattern to match chapter titles (Roman numerals followed by title)
chapter_pattern = re.compile(r"\n([IVXLCDM]+)\.\s([^\n]+)\n")

# Find all matches (chapter titles)
matches = list(chapter_pattern.finditer(text))

# Split the text into chapters
chapters = []
start = matches[0].end() if matches else 0  # Start at first chapter if found

for i in range(len(matches)):
    end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

    chapter_title = matches[i].group(2)  # Extract chapter title (ignore Roman numeral)
    chapter_content = text[matches[i].end():end].strip()

    # Remove subchapter numbers (standalone Roman numerals followed by a newline)
    chapter_content = re.sub(r"^\n?[IVXLCDM]+\.\n+", "", chapter_content, flags=re.MULTILINE).strip()

    # Store as "Chapter Title\nContent"
    chapters.append(f"{chapter_title}\n\n{chapter_content}")

# Print the first 300 characters of each chapter
for i, chapter in enumerate(chapters, 1):
    print(f"Chapter {i}: {chapter[:300]}...\n")


Chapter 1: A SCANDAL IN BOHEMIA

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
wer...

Chapter 2: THE RED-HEADED LEAGUE

I had called upon my friend, Mr. Sherlock Holmes, one day in the
 autumn of last year and found him in deep conversation with a very
 stout, florid-faced, elderly gentleman with fiery red hair. With an
 apology for my intrusion, I was about to withdraw when Holmes pulled
 me a...

Chapter 3: A CASE OF IDENTITY

“My dear fellow,” said Sherlock Holmes as we sat on either side of the
fire in his lodgings at Baker Street, “life is infinitely stranger than
anything which the mind of man could invent. We would not dare to
conceive the things which are really mere commonplaces of existence. If...

Chapter 4: THE BOSCOMBE VALLEY MYSTERY

We were seat

In [5]:
new_text = '\n'.join(chapters)
print(new_text[:1500])

A SCANDAL IN BOHEMIA

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. He
was, I take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in a
false position. He never spoke of the softer passions, save with a gibe
and a sneer. They were admirable things for the observer—excellent for
drawing the veil from men’s motives and actions. But for the trained
reasoner to admit such intrusions into his own delicate and finely
adjusted temperament was to introduce a distracting factor which might
throw a doubt upon all his mental results. Grit in a sensitive
instrument, or a crack in one of his own high-power lenses, would not
be more distu

In [6]:
with open('dataset_clean.txt', 'w') as f:
  f.write(new_text)

Observando el dataset limpio, vemos que
- `\n\n` (consecutive endline)
- `_the_` (emphasis)
- `“Seven!”` (punctuation)
- `“Absolutely?”` (punctuation)
- `John’s` (punctuation)
- `won’t` (contractions)
- `half-dragged` (hyphened compound-words)
- `“‘The Church of St. Monica,` (abbreviation)
- `\t\t“MY DEAR MR. SHERLOCK HOLMES` (tabs/spaces)
- `_née_` (tilde)
- `“‘Never.’` (quotes)
- `“THE RED-HEADED LEAGUE IS DISSOLVED. October 9, 1890.”` (numbered date)
- `sitting-room`
- `“‘December 22nd. Twenty-four geese at 7_s_. 6_d_.’”`

Deberíamos aplicar un preprocesamiento al texto.

Una práctica recomendada es pasar todo a minúscula

In [31]:
!pip install contractions dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.2 MB/s eta 0:00:00


In [32]:
import re
import contractions
import dateparser

def preprocess_sherlock(text):
    # 1. Normalize Line Breaks & Whitespace
    text = re.sub(r'\n+', '\n', text)  # Reduce multiple newlines to one
    text = re.sub(r'\s+', ' ', text)   # Replace multiple spaces/tabs with a single space

    # 2. Remove Emphasis (_the_ → the)
    text = re.sub(r'_([^_]+)_', r'\1', text)

    # 3. Standardize Quotes & Punctuation
    text = text.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    # 4. Expand Contractions (won’t → will not)
    text = contractions.fix(text)

    # 5. Normalize Hyphenated Words (half-dragged → half_dragged)
    text = re.sub(r'(\w+)-(\w+)', r'\1_\2', text)

    # 6. Normalize Abbreviations (St. → Saint)
    abbreviations = {"St.": "Saint", "Dr.": "Doctor"}
    for abbr, full in abbreviations.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)

    # 7. Convert Numbered Dates (October 9, 1890 → 1890-10-09)
    text = re.sub(r'(\b[A-Za-z]+ \d{1,2}, \d{4})',
                  lambda m: dateparser.parse(m.group(0)).strftime('%Y-%m-%d'), text)

    # 8. Normalize Money & Measurements (7_s_. 6_d_. → 7s 6d)
    text = re.sub(r'(\d+)_s_\.? (\d+)_d_\.?', r'\1s \2d', text)

    return text


In [34]:
with open('dataset_clean_preprocessed.txt', 'w') as f:
  f.write(preprocess_sherlock(new_text))

## 1 - Preprocesamiento

In [35]:
import pandas as pd
df = pd.read_csv('dataset_clean_preprocessed.txt', sep='/n', header=None)
# df.head()

<ipython-input-35-12cfb9028a78>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('dataset_clean_preprocessed.txt', sep='/n', header=None)


In [36]:
print("Cantidad de lineas:", df.shape[0])

Cantidad de lineas: 1


In [37]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))



In [38]:
sentence_tokens[:2]

[['a',
  'scandal',
  'in',
  'bohemia',
  'to',
  'sherlock',
  'holmes',
  'she',
  'is',
  'always',
  'the',
  'woman',
  'i',
  'have',
  'seldom',
  'heard',
  'him',
  'mention',
  'her',
  'under',
  'any',
  'other',
  'name',
  'in',
  'his',
  'eyes',
  'she',
  'eclipses',
  'and',
  'predominates',
  'the',
  'whole',
  'of',
  'her',
  'sex',
  'it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler',
  'all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind',
  'he',
  'was',
  'i',
  'take',
  'it',
  'the',
  'most',
  'perfect',
  'reasoning',
  'and',
  'observing',
  'machine',
  'that',
  'the',
  'world',
  'has',
  'seen',
  'but',
  'as',
  'a',
  'lover',
  'he',
  'would',
  'have',
  'placed',
  'himself',
  'in',
  'a',
  'false',
  'position',
  'he',
  'never',
 

### 2 - Crear los vectores (word2vec)

In [39]:
from gensim.models.callbacks import CallbackAny2Vec

# class LossCallback(CallbackAny2Vec):
#     """
#     Callback to print loss after each epoch in Gensim training.
#     """
#     def __init__(self):
#         self.epoch = 0
#         self.loss_previous_step = 0

#     def on_epoch_end(self, model):
#         if hasattr(model, 'running_training_loss'):  # Newer gensim versions
#             loss = model.running_training_loss
#         else:
#             loss = None  # Fallback if attribute is missing (should not happen in modern gensim)

#         if loss is not None:
#             print(f'Loss after epoch {self.epoch}: {loss - self.loss_previous_step}')
#             self.loss_previous_step = loss
#         else:
#             print(f'Loss after epoch {self.epoch}: Loss tracking is unavailable')

#         self.epoch += 1

class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [40]:
from gensim.models import Word2Vec
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [41]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [42]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1


In [43]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2052


3 - Entrenar embeddings

In [44]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 272221.15625
Loss after epoch 1: 138067.21875
Loss after epoch 2: 123256.5625
Loss after epoch 3: 119673.25
Loss after epoch 4: 117552.0
Loss after epoch 5: 118267.6875
Loss after epoch 6: 117914.9375
Loss after epoch 7: 111598.9375
Loss after epoch 8: 108608.5
Loss after epoch 9: 108995.875
Loss after epoch 10: 107929.0
Loss after epoch 11: 107817.5
Loss after epoch 12: 107645.25
Loss after epoch 13: 107047.875
Loss after epoch 14: 107318.375
Loss after epoch 15: 107526.5
Loss after epoch 16: 107509.75
Loss after epoch 17: 89113.875
Loss after epoch 18: 87625.0
Loss after epoch 19: 87397.75


(200000, 2117600)

### 4 - Ensayar

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sherlock"], topn=10)

[('jabez', 0.9801268577575684),
 ('wilson', 0.9648646116256714),
 ('said', 0.9618110656738281),
 ('duncan', 0.9512567520141602),
 ('merryweather', 0.9472882747650146),
 ('mr', 0.9443387985229492),
 ('ross', 0.9239485263824463),
 ('holmes', 0.9206176400184631),
 ('now', 0.9149553179740906),
 ('asked', 0.9149285554885864)]